##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [7]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hand Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect hand landmarks from images.

## Preparation

Let's start with installing MediaPipe.

In [8]:
# !pip install -q mediapipe

Then download an off-the-shelf model bundle. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker#models) for more information about this model bundle.

In [9]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Visualization utilities

In [13]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
from math import sqrt

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def calculate_distance(puntos,pos):
  dist = sqrt((puntos[pos].x - puntos[0].x)**2 + (puntos[pos].y - puntos[0].y)**2 + (puntos[pos].z - puntos[0].z)**2)
  return dist

def pintar_texto(annotated_image, text_x, text_y, texto):
  cv2.putText(annotated_image, texto,
          (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
          FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    #Detectar si es piedra, papel o tigera
    puntos = []
    for landmark in hand_landmarks:
      puntos.append(landmark)

    if(calculate_distance(puntos,8) > calculate_distance(puntos,6) and calculate_distance(puntos,12) > calculate_distance(puntos,10)):

      if (calculate_distance(puntos,16) > calculate_distance(puntos,14) and calculate_distance(puntos,20) > calculate_distance(puntos,18)): 

        pintar_texto(annotated_image,text_x,text_y,"Papel")

      elif(calculate_distance(puntos,16) < calculate_distance(puntos,14) and calculate_distance(puntos,20) < calculate_distance(puntos,18)):

        pintar_texto(annotated_image,text_x,text_y,"Tijera")

    elif(calculate_distance(puntos,8) < calculate_distance(puntos,6) and calculate_distance(puntos,12) < calculate_distance(puntos,10) and 
       calculate_distance(puntos,16) < calculate_distance(puntos,14) and calculate_distance(puntos,20) < calculate_distance(puntos,18)):
      
      pintar_texto(annotated_image,text_x,text_y,"Piedra")



    # #Papel
    # if(calculate_distance(puntos,12) > calculate_distance(puntos,11) and calculate_distance(puntos,16) > calculate_distance(puntos,15) and 
    #    calculate_distance(puntos,20) > calculate_distance(puntos,19)):
    #   pintar_texto(annotated_image,text_x,text_y,"Papel")

    # #Tijera
    # if(calculate_distance(puntos,8) > calculate_distance(puntos,7) and 
    #    calculate_distance(puntos,12) > calculate_distance(puntos,11) and 
    #    calculate_distance(puntos,16) < calculate_distance(puntos,13) and 
    #    calculate_distance(puntos,20) < calculate_distance(puntos,17)):
    #   pintar_texto(annotated_image,text_x,text_y,"Tijera")


    # Draw handedness (left or right hand) on the image.
    # cv2.putText(annotated_image, f"{handedness[0].category_name}",
    #             (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
    #             FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

## Running inference and visualizing the results

Here are the steps to run hand landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python) to learn more about configuration options that this solution supports.


In [15]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
import time
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options, running_mode=vision.RunningMode.VIDEO,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

# Abrir la camara
cap = cv2.VideoCapture(0)

current_timestamp_ms = time.time()

while (cap.isOpened()):
    ret, frame = cap.read()
    img1 = frame

    if ret == True:
        #Mostrar el video
        
        # Bucle lectura imagen
        # STEP 3: Load the input image.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img1)

        # STEP 4: Detect hand landmarks from the input image.
        frame_timestamp_ms = round((time.time() - current_timestamp_ms) * 1000)
        hand_landmarker_result = detector.detect_for_video(mp_image, frame_timestamp_ms)

        # STEP 5: Process the classification result. In this case, visualize it.
        annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)
        # plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

        cv2.imshow('', cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))

        if cv2.waitKey(10) & 0xFF == ord('q'): break
cap.release()
cv2.destroyAllWindows()